In [ ]:
from sympy import symbols, sqrt, Eq, simplify, expand, solveset, collect
x, y, z = symbols('x y z')
r = symbols('r')
c, k = symbols('c k')
A, B, C, D, E, F, G, H, I, J = symbols('A B C D E F G H I J')

## Lens design formula: canonical quadric form

In [ ]:
## Lens design surface equation for 'Standard'.
eq_ld = Eq(z, c * r**2 / (1 + sqrt(1 - (1+k) * c**2 * r**2)))
eq_ld

In [ ]:
## General quadric formula
eq_quad = Eq(A*x**2 + B*y**2 + C*z**2 + D*x*y + E*y*z + \
      F*x*z + G*x + H*y + I*z + J, 0)
eq_quad

In [ ]:
## Lens design formula as canonical quadric
eq_os_quad = eq_rev.subs([(C, (k+1)), (I, -2/c), (J, 0)])
eq_os_quad

In [ ]:
## Getting back to the original 'z =' form.
solveset(eq_os_quad, z).simplify()

## Interactive plotly

In [ ]:
import numpy as np
import plotly.graph_objects as go

In [ ]:
x = np.linspace(-5.0, 5.0, num=50); y = x
xs, ys = np.meshgrid(x, y)

sphere_rad = -30
z_sphere = sphere_rad * (1 - np.sqrt(1- (xs**2 + ys**2)/(sphere_rad**2)))

k = -3 # Conic constant
c = 1 / sphere_rad
r = np.sqrt(xs**2 + ys**2)

## Two solution sheets, the one we choose is z_std_1.
z_std_1 = c * r**2 / (1 + np.sqrt(1 - (k+1) * c**2 * r**2))
z_std_2 = c * r**2 / (1 - np.sqrt(1 - (k+1) * c**2 * r**2))

sphere = go.Surface(x = x, y = y, z = z_sphere, opacity=0.5,
                    name='sphere')
std_1 = go.Surface(x = x, y = y, z = z_std_1, opacity=0.5,
                   name='sheet 1')
std_2 = go.Surface(x = x, y = y, z = z_std_2, opacity=0.1,
                   name='sheet 2')

fig = go.Figure(data=[sphere, std_1])
fig.update_layout(scene = {'aspectratio': {'x': 1, 'y': 1, 'z': 0.5}})
fig.show()